In [10]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sys
sys.path.append('../Task 1/')
from efficient_apriori import apriori
from improved_apriori import Improved_Apriori
import json
import time
import os
import itertools
import ast
from tqdm import tqdm
import collections
import math
import random
import ijson

In [11]:
# Process the dataset by chunks into username and the list of book the user watched
def process_book_chunk(df, carry_over):
    if carry_over is not None:
        df = pd.concat([carry_over, df])
    groups = df.groupby('User_id')['Title'].apply(list)
    last_user = df.iloc[-1]['User_id']
    if last_user in groups:
        carry_over = df[df['User_id'] == last_user]
        groups = groups.drop(last_user)
    else:
        carry_over = None
    return groups, carry_over

In [12]:
def isNotNan(array):
    for item in array:
        if not np.isnan(item):
            return False
    return True;

In [13]:
carry_over = None
chunksize = 10000 # adjust this value depending on your available memory
csv_file_name = "dataset/Books_rating.csv"
book_dict = {}
# Might have to figure out a way to shuffle the dataset 
if(not os.path.exists('dataset/processed_book_output.txt')):
    with open('dataset/processed_book_output.txt', 'w') as f:
        for chunk in pd.read_csv(csv_file_name, chunksize=chunksize):
            chunk.dropna(how='any',subset=['Title'], inplace=True)
            groups, carry_over = process_book_chunk(chunk, carry_over)
            for user, book_list in groups.items():
                if user != "nan" and len(book_list) > 1:
                    f.write(f'{user} {book_list}\n')

        # don't forget to process the last carry_over
        if carry_over is not None:
            groups, _ = process_book_chunk(carry_over, None)
            for user, book_list in groups.items():
                f.write(f'{user} {book_list}\n')

In [14]:
# Shuffle the text in chunks 
csv_file = 'dataset/Books_rating.csv'

def shuffle_large_file(file_name, output_file_name, chunk_size):
    with open(file_name, 'r') as f:
        while True:
            lines = list(itertools.islice(f, chunk_size))
            if not lines:
                break
            random.shuffle(lines)
            with open(output_file_name, 'a') as out:
                out.write(''.join(lines))


# Call the function with your parameters
if(not os.path.exists('dataset/processed_book_output_shuffled.txt')):
    shuffle_large_file('dataset/processed_book_output.txt', 'dataset/processed_book_output_shuffled.txt', 3000000)

In [15]:
def read_file_in_partitions(file_path, partition_size):
    with open(file_path, 'r') as file:
        partition = []
        for line in file:
            partition.append(line)
            if len(partition) >= partition_size:
                yield partition
                partition = []
        if partition:  # yield any remaining lines
            yield partition

In [16]:
# Global variable to get the counts of all itemsets
global_counts = {}
def generate_global_counts(partition, global_candidates):

    # For 1th itemset, generate the transaction id list for the ith partition 
    transaction_id_dict = collections.defaultdict(list)
    for transaction_id in partition:
        for item in partition[transaction_id]:
            item_tuple = (item,)
            transaction_id_dict[item_tuple].append(transaction_id)

    # Filter based on the global candidates formed
    transaction_ids_dict = {item: transaction_ids for item, transaction_ids in transaction_id_dict.items() if item in global_candidates[1]}

    # Get the global count of all 1th itemset
    for item in transaction_id_dict:
        if(len(item) not in global_counts):
            global_counts[len(item)] = {}
        if(item not in global_counts[len(item)]):
            global_counts[len(item)][item] = len(transaction_id_dict[item])
        else:
            global_counts[len(item)][item] += len(transaction_id_dict[item])

    # Extend to find global count of all nth itemset from the global candidates
    for i in tqdm(range(1, len(global_candidates))):
        for itemset in global_candidates[i+1]:
            transaction_ids = set(transaction_id_dict[(itemset[0],)])
            for i in range(1, len(itemset)):
                # We are only interested in the transactions where all items in itemset is present
                transaction_ids = transaction_ids.intersection(set(transaction_ids_dict.get((itemset[i],), {})))
            if(len(itemset) not in global_counts):
                global_counts[len(itemset)] = {}

            if(itemset not in global_counts[len(itemset)]):
                global_counts[len(itemset)][itemset] = len(transaction_ids)
            else:
                global_counts[len(itemset)][itemset] += len(transaction_ids)



In [49]:
file_path = 'dataset/processed_book_output_shuffled.txt'

size_of_data = sum(1 for line in open(file_path))
min_support=0.0001
partition_size = 50000
num_partitions = size_of_data// partition_size
print(num_partitions)
partition_candidates = []
global_candidates = collections.defaultdict(list)
# Step 1: Partitioning
global_min_support = math.ceil((min_support*size_of_data)/num_partitions)
for i, partition in enumerate(read_file_in_partitions(file_path, partition_size)):
    print(f'Partition {i+1}:')
    dict_book = {}
    for line in partition:
        user, book_list_str = line.strip().split(' ', 1)
        #print(book_list_str)
        book_list = ast.literal_eval(book_list_str)
        dict_book[user] = book_list
   
    improved_apriori = Improved_Apriori(dict_book, min_support=min_support, min_confidence=1, verbose=2)
    # Step 2: Retreieve frequent itemset per partition
    partition_frequent_itemset = improved_apriori.apriori()
    # Efficient Apriori for sanity check
    #partition_frequent_itemset, _ = apriori(list(dict_book.values()), min_support = min_support, verbosity=2)

    # Form the global candidate set from the large itemset in each partitions
    # In this space, we ignore the count of itemset in each partition as they are not useful in our global support count
    # All they do is just show the itemset was large enough in the current partition
    # Merging Phase
    for level, itemset in partition_frequent_itemset.items():
        for key in itemset.keys():
            if(key not in global_candidates[level]):
                global_candidates[level].append(key)

2
Partition 1:
Found 15617 candidate itemsets from 1st Level
Found 1991 frequent itemsets from 1th item candidate sets
Found 1981045 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 0.09640741348266602


100%|████████████████████████████████████████████████████████████████████| 1981045/1981045 [00:05<00:00, 352542.70it/s]


Found 393 frequent itemsets from 2th item candidate sets
Found 24 candidates for 3th item candidate sets
Time taken to find 3th item candidate sets: 0.026041269302368164


100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


Found 19 frequent itemsets from 3th item candidate sets
Found 2 candidates for 4th item candidate sets
Time taken to find 4th item candidate sets: 0.0004787445068359375


...

100%|██████████████████████████████████████████████████████████████████████| 139785/139785 [00:00<00:00, 236406.08it/s]


Found 139785 frequent itemsets from 6th item candidate sets
Found 352613 candidates for 7th item candidate sets
Time taken to find 7th item candidate sets: 3349.8560738563538


100%|██████████████████████████████████████████████████████████████████████| 352613/352613 [00:01<00:00, 209719.93it/s]


Found 352613 frequent itemsets from 7th item candidate sets


KeyboardInterrupt: 

In [45]:
# Now we have to read the lines in chunks for our disk-based operations
min_support_count = min_support * size_of_data
for i, partition in enumerate(read_file_in_partitions(file_path, partition_size)):
    print(f'Partition {i+1}:')

    # Hold the partition data in main memory
    dict_book = {}
    for line in partition:
        user, book_list_str = line.strip().split(' ', 1)
        book_list = ast.literal_eval(book_list_str)
        dict_book[user] = book_list
    # Pure disk based implementation would probably require us to save the global candidates in disk 
    generate_global_counts(dict_book, global_candidates)
    
        

Partition 1:


100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


Partition 2:


100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


Partition 3:


...

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2840.39it/s]


Partition 22:


100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


Partition 23:


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 209.79it/s]


In [50]:
global_frequent_itemsets = {level: {itemset: count for itemset, count in itemsets.items() if count >= min_support_count} for level, itemsets in global_counts.items()}
global_frequent_itemsets

{1: {('Lady Susan',): 171,
  ('Lord of the Flies, a Novel',): 356,
  ('Great Expectations',): 6287,
  ('Pride and Prejudice',): 4561,
  ('Adventures of Huckleberry Finn',): 458,
  ('The Poisonwood Bible',): 374,
  ("Cat's Cradle",): 3336,
  ('Catch 22',): 661,
  ("Cat's cradle (A Dell book)",): 165,
  ('An inquiry into the nature and causes of the wealth of nations,',): 932,
  ('An inquiry into the nature and causes of the wealth of nations',): 2695,
  ('Of Mice And Men',): 507,
  ('A Tree Grows in Brooklyn',): 661,
  ('Brave New World',): 1994,
  ('To Kill a Mocking Bird',): 3656,
  ('To Kill a Mockingbird',): 3705,
  ('And then there were none',): 258,
  ('Cry, the beloved country',): 135,
  ('Night',): 647,
  ('Lion, the Witch, and the Wardrobe',): 123,
  ('The Hobbit',): 4595,
  ('Fahrenheit 451',): 1628,
  ('Monte Cristo,',): 231,
  ('Of Mice and Men',): 2548,
  ('Dandelion Wine',): 235,
  ('Trojan Odyssey: A Dirk Pitt Novel',): 1026,
  ('Dracula (G. K. Hall (Large Print))',): 365

In [51]:
for level in global_frequent_itemsets:
    print(len(global_frequent_itemsets[level]))

856
33
1
0


In [52]:
data = pd.read_csv('dataset/Books_rating.csv')
data = data[['User_id', 'Title']]
grouped_data = data.groupby('User_id')['Title'].apply(list)
grouped_data = grouped_data.to_dict()

In [53]:
improved_apriori = Improved_Apriori(grouped_data, min_support=min_support, min_confidence=1, verbose=2)
frequent_book_set = improved_apriori.apriori()

Found 206712 candidate itemsets from 1st Level


TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:
itemset, _ = apriori(list(grouped_data.values()), min_support = min_support, verbosity=2)

In [ ]:
frequent_book_set

In [ ]:
itemset

In [55]:
f = open("dataset/books_frequent_itemsets.txt", "w")
f.write(str(global_frequent_itemsets))
f.close()